# Week 1 Challenge 
Guess the data from a given received signal file and FIR of the channel 

Parameters:
- Constellations:
    - Bytes were mapped to QPSK constellations 2 bits at a time  (00, 01, 11, 10)
- IDFT:
    - size of 1024
    - Cyclic prefix of length 32
    - 2 bit groups were added at the end of the data to complete an integer factor of 511
    - the resulting blocks of 1056 values were concatenated into one long signal


### 0.1 Import Library

In [1]:
import pandas as pd
import numpy as np

### 1.1 Import channel and files

In [2]:
FIR = np.genfromtxt('channel.csv', delimiter=',')

In [3]:
file1 = np.genfromtxt("file1.csv", delimiter=',')

### 1.2 Check the length of the file

In [4]:
len(file1)

1003200

In [5]:
len(file1)/(1024+32)

950.0

### 2. Remove Cyclic Prefix

#### 2.1 First transform the 1D array into 2D array 
    
    where each row is of 1056 long and there is 950 such rows

In [6]:
file1_2d = file1.reshape(-1,1056)

In [7]:
file1_2d.shape

(950, 1056)

#### 2.2 remove cyclic prefix
    Equivalent to delte columns from index 1024 to 1055(last)

In [8]:
remove = np.arange(32) + 1024
remove

array([1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045,
       1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055])

In [9]:
file1_without_cp = np.delete(file1_2d,remove,axis=1)

In [10]:
file1_without_cp.shape

(950, 1024)

### 3. Apply DFT

In [11]:
file1_dft = np.array( [np.fft.fft(file1_without_cp[i,:]) for i in range(file1_without_cp.shape[0]) ] ) 

In [12]:
file1_dft.shape

(950, 1024)

#### 4. Apply channel equalization 

In [29]:
# impulse response of the channel
channel_fft = np.fft.fft(FIR, 1024)

In [30]:
Equalized_output = file1_dft / H

In [31]:
out = Equalized_output

### 5. Maximum likelihood estimate for constellations

$$(0,0) \rightarrow  1+1j$$

$$(0,1) \rightarrow  -1+1j$$

$$(1,1) \rightarrow  -1+-1j$$

$$(1,0) \rightarrow  1-1j$$

In [32]:
ofdm_frames = out
print(ofdm_frames.shape)
data = ''
prefix = 32
N = 1024
for i in range(len(ofdm_frames)):
    frame_prefix = ofdm_frames[i][prefix:] # remove cp
    frame_dft = np.fft.fft(frame_prefix, n=N) # 
    for i in range(1, 512): # only useful info bits 1-511
        element = frame_dft[i] / channel_fft[i]
        if np.real(element) >= 0:
            if np.imag(element) >= 0:
                data += '00'
            else:
                data += '10'
        else:
            if np.imag(element) >= 0:
                data += '01'
            else:
                data += '11'

(950, 1024)


In [34]:
len(data)

970900

In [36]:
950*1024 - 970900

1900

In [37]:
1003200

1003200

In [38]:
1003200 - 970900

32300

In [40]:
from bitarray import bitarray

data_bytes = bitarray(data)

In [41]:
len(data_bytes)

970900

In [43]:
len(data_bytes.tobytes()[:])

121363